In [1]:
import pandas as pd

**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток: geo_data = {...}

Результат классификации запишите в отдельный столбец region.

In [2]:
kw=pd.read_csv('keywords.csv')
geo_data={'Центр':['москва','тула','ярославль'],
          'Северо-Запад':['петербург','псков','мурманск'],
          'Дальний Восток':['владивосток','сахалин','хабаровск']}

In [3]:
def geo_def(row):
    for key in geo_data:
        if any(x in row['keyword'] for x in geo_data[key]): geo_str=key
        else: geo_str='undefined'
    return geo_str

In [4]:
kw['region']=kw.apply(geo_def, axis=1)
kw[kw['region']!='undefined'].head()

,keyword,shows,region
1063,фарпост владивосток,176951,Дальний Восток
2175,дром хабаровск,98361,Дальний Восток
3206,дром владивосток,73084,Дальний Восток
3275,100 сп владивосток,76418,Дальний Восток
3687,авито хабаровск,63736,Дальний Восток


**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [38]:
movies=pd.read_csv('ml-latest-small/movies.csv')
ratings=pd.read_csv('ml-latest-small/ratings.csv')
#сделаем табличку со средними оценками у фильмов
movie_rating=movies.merge(ratings.groupby('movieId').mean()['rating'].reset_index(),on='movieId', how='left')

In [39]:
movie_rating.head()

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615
4,5,Father of the Bride Part II (1995),Comedy,3.267857


In [40]:
def movie_class(row):
    y=float(row['rating'])
    if y<=2: return 'Низкий рейтинг'
    elif y<=4.5: return 'Средний рейтинг'
    elif y>4.5: return 'Высокий рейтинг'

In [42]:
movie_rating['class']=movie_rating.apply(movie_class,axis=1)
movie_rating.head()

,movieId,title,genres,rating,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,Средний рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,Средний рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,Средний рейтинг
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,Средний рейтинг
4,5,Father of the Bride Part II (1995),Comedy,3.267857,Средний рейтинг


**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [44]:
#Подобное было на 5й лекции
ratings_grouped = ratings.groupby('userId').agg([min, max]).reset_index()
ratings_grouped['diff'] = (ratings_grouped['timestamp']['max'] - ratings_grouped['timestamp']['min'])/24/3600
ratings_grouped.head()

userId movieId        rating        timestamp                  diff
             min    max    min  max         min         max          
0      1      31   3671    1.0  4.0  1260759108  1260759205  0.001123
1      2      10    720    1.0  5.0   835355395   835356246  0.009850
2      3      60  84236    2.0  5.0  1298861589  1298932787  0.824051
3      4      10   4006    1.0  5.0   949778714   949982274  2.356019
4      5       3  48385    1.5  5.0  1163373044  1163375145  0.024317

In [48]:
#А это на 6й
grouped=ratings.groupby('userId').count().reset_index()
ratings_filtered = grouped[grouped['timestamp'] >= 100]
ratings_grouped[ratings_grouped['userId'].isin(ratings_filtered['userId'].tolist())]['diff'].mean()

455.22857137198986

**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [75]:
years=range(1950,2010)
def production_year(row):
    for i in years: 
        if str(i) in str(row['title']): return str(i)

In [77]:
#воспользуемся предыдущей таблицей movie_rating
movie_rating['year']=movie_rating.apply(production_year, axis=1)
movie_rating.head()

,movieId,title,genres,rating,class,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,Средний рейтинг,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,Средний рейтинг,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,Средний рейтинг,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,Средний рейтинг,1995
4,5,Father of the Bride Part II (1995),Comedy,3.267857,Средний рейтинг,1995


In [81]:
movie_rating.groupby('year')[['rating']].mean().sort_values(by='rating',ascending=False).head(10)

,rating
year,
1951,3.906653
1952,3.802318
1962,3.772962
1953,3.738878
1957,3.693322
1973,3.688212
1971,3.643227
1969,3.604509
1972,3.585295
